In [5]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

In [13]:
sc.install_pypi_package('s3fs')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/72/5c/ec84c7ec49fde2c3b0d885ecae4504fa40fc77fef7684e9f2939c50f9b94/s3fs-0.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/39/8b/250778bc5fd4b9a91d7916f8729b346931be419052130548617139247ba6/boto3-1.10.41-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/49/e0/a9a53656126a635120c0f7eaee3ff3b26a4a08596d10e7192fd91ce19e5c/botocore-1.13.41-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/dd/1f/7028dacd3c28f34ce48130aae73a88fa5cc27b6b0e494fcf2739f7954d9d/fsspec-0.6.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/8a/1fc3dba0c4923c2a76e1ff0d52b305c44606da63f718d14d3231e21c51b0/s3transfer-0.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/22/cd/a6aa959dca619918ccb55023b4cb151949c64d4d5d55b3f4ffd7eee0c6e8/docutils-0.15.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62fa

## Reading mentions database

In [ ]:
df_mention = spark.read.parquet('s3://bdcc-lab/mention.parquet')

In [ ]:
df_mention = df_mention.drop(*['EventTimeDate', 'MentionSourceName', 'MentionIdentifier', 'InRawText', 'Confidence', 'MentionDocTranslationInfo', 'Extras'])

## Reading events database

In [ ]:
events = spark.read.csv('s3://gdelt-open-data/v2/events/2019*.csv',
                     sep='\t',
                     header=None)

In [ ]:
cols_events= ['GlobalEventID',
             'Day',
             'MonthYear',
             'Year',
             'FractionDate', 
             'Actor1Code',
             'Actor1Name',
             'Actor1CountryCode',
             'Actor1KnownGroupCode',
             'Actor1EthnicCode',
             'Actor1Religion1Code',
             'Actor1Religion2Code',
             'Actor1type1Code',
             'Actor1type2Code',
             'Actor1type3Code',
             'Actor2Code',
             'Actor2Name',
             'Actor2CountryCode',
             'Actor2KnownGroupCode',
             'Actor2EthnicCode',
             'Actor2Religion1Code',
             'Actor2Religion2Code',
             'Actor2type1Code',
             'Actor2type2Code',
             'Actor2type3Code',
             'IsRootEvent',
              'EventCode',
              'EventBaseCode',
              'EventRootCode',
              'QuadClass',
              'GoldsteinScale',
              'NumMentions',
              'NumSources',
              'NumArticles',
              'AvgTone',
              'Actor1Geo_Type',
              'Actor1Geo_Fullname',
              'Actor1Geo_CountryCode',
              'Actor1Geo_ADM1Code',
              'Actor1Geo_ADM2Code',
              'Actor1Geo_Lat',
              'Actor1Geo_Long',
              'Actor1Geo_FeatureID',
              'Actor2Geo_Type',
              'Actor2Geo_Fullname',
              'Actor2Geo_CountryCode',
              'Actor2Geo_ADM1Code',
              'Actor2Geo_ADM2Code',
              'Actor2Geo_Lat',
              'Actor2Geo_Long',
              'Actor2Geo_FeatureID',
              'ActionGeo_Type',
              'ActionGeo_Fullname',
              'ActionGeo_CountryCode',
              'ActionGeo_ADM1Code',
              'ActionGeo_ADM2Code',
              'ActionGeo_Lat',
              'ActionGeo_Long',
              'ActionGeo_FeatureID',
              'DATEADDED',
              'SOURCEURL'
             ]

for i, col_names in enumerate(cols_events):
    events = events.withColumnRenamed('_c'+str(i), col_names)

In [ ]:
events.write.parquet('s3://bdcc-lab/all_events.parquet')

In [ ]:
events.createOrReplaceTempView('events')

### Filtering to Philippines only

In [ ]:
phil_events = spark.sql('''select * from events 
            where Actor1CountryCode == "RP" or Actor1Geo_CountryCode == "RP" or Actor2CountryCode == "RP" or 
            Actor2Geo_CountryCode == "RP" or ActionGeo_CountryCode == "RP"
            ''')

In [10]:
phil_events = phil_events.withColumn('mention_source', phil_events['NumMentions']/phil_events['NumSources'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
phil_events = phil_events.withColumn('mention_article', phil_events['NumMentions']/phil_events['NumArticles'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
phil_events.write.parquet('s3://bdcc-lab/phil_events_final.parquet')

In [5]:
phil_events = phil_events[[
    'GlobalEventID',
         'Day',
         'Actor1Code',
         'Actor1CountryCode',
         'Actor1KnownGroupCode',
         'Actor1EthnicCode',
         'Actor1Religion1Code',
         'Actor1Religion2Code',
         'Actor1type1Code',
         'Actor1type2Code',
         'Actor1type3Code',
         'Actor2Code',
         'Actor2CountryCode',
         'Actor2KnownGroupCode',
         'Actor2EthnicCode',
         'Actor2Religion1Code',
         'Actor2Religion2Code',
         'Actor2type1Code',
         'Actor2type2Code',
         'Actor2type3Code',
         'IsRootEvent',
          'EventCode', 
        'EventBaseCode',
        'EventRootCode',
         'QuadClass',
         'NumMentions',
         'NumSources',
         'NumArticles',
        'AvgTone'
]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
phil_events.write.parquet('s3://bdcc-lab/phil_events_final.parquet')

## Reading mentions parquet dataset

In [3]:
df_mention = spark.read.parquet('s3://bdcc-lab/mention.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_mention = df_mention.drop(*['EventTimeDate', 'MentionSourceName', 'MentionIdentifier', 'InRawText', 'Confidence', 'MentionDocTranslationInfo', 'Extras'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reading philippine events parquet dataset

In [5]:
phil_events = spark.read.parquet('s3://bdcc-lab/phil_events_final.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Filtering the features

In [6]:
phil_events = phil_events[[
    'GlobalEventID',
         'Day',
         'Actor1Code',
         'Actor1CountryCode',
         'Actor1KnownGroupCode',
         'Actor1EthnicCode',
         'Actor1Religion1Code',
         'Actor1Religion2Code',
         'Actor1type1Code',
         'Actor1type2Code',
         'Actor1type3Code',
         'Actor2Code',
         'Actor2CountryCode',
         'Actor2KnownGroupCode',
         'Actor2EthnicCode',
         'Actor2Religion1Code',
         'Actor2Religion2Code',
         'Actor2type1Code',
         'Actor2type2Code',
         'Actor2type3Code',
         'IsRootEvent',
          'EventCode', 
        'EventBaseCode',
        'EventRootCode',
         'QuadClass',
         'NumMentions',
         'NumSources',
         'NumArticles',
        'AvgTone'
]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Feature engineering

In [7]:
phil_events = phil_events.withColumn('mention_source', phil_events['NumMentions']/phil_events['NumSources'])
phil_events = phil_events.withColumn('mention_article', phil_events['NumMentions']/phil_events['NumArticles'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
phil_events.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GlobalEventID: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1type1Code: string (nullable = true)
 |-- Actor1type2Code: string (nullable = true)
 |-- Actor1type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCode: string (nullable = true)
 |-- Actor2EthnicCode: string (nullable = true)
 |-- Actor2Religion1Code: string (nullable = true)
 |-- Actor2Religion2Code: string (nullable = true)
 |-- Actor2type1Code: string (nullable = true)
 |-- Actor2type2Code: string (nullable = true)
 |-- Actor2type3Code: string (nullable = true)
 |-- IsRootEvent: string (nullable = true)
 |--

In [9]:
phil_events.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+----------+-----------------+--------------------+----------------+-------------------+-------------------+---------------+---------------+---------------+-----------+---------+-------------+-------------+---------+-----------+----------+-----------+-----------------+--------------+---------------+
|GlobalEventID|     Day|Actor1Code|Actor1CountryCode|Actor1KnownGroupCode|Actor1EthnicCode|Actor1Religion1Code|Actor1Religion2Code|Actor1type1Code|Actor1type2Code|Actor1type3Code|Actor2Code|Actor2CountryCode|Actor2KnownGroupCode|Actor2EthnicCode|Actor2Religion1Code|Actor2Religion2Code|Actor2type1Code|Actor2type2Code|Actor2type3Code|IsRootEvent|EventCode|EventBaseCode|EventRootCode|QuadClass|NumMentions|NumSources|NumArticles|          AvgTone|mention_source|mention_article|
+-------------+--------+----------+-------

## Merging files

In [10]:
#mentions_count = 9250645
#phile_events_count = 195826
#innerjoin = 44069

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
sample_joined = df_mention.join(phil_events, "GlobalEventID")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
sample_joined.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

44069

In [11]:
sample_joined.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GlobalEventID: string (nullable = true)
 |-- MentionTimeDate: string (nullable = true)
 |-- MentionType: string (nullable = true)
 |-- SentenceID: string (nullable = true)
 |-- Actor1CharOffset: string (nullable = true)
 |-- Actor2CharOffset: string (nullable = true)
 |-- ActionCharOffset: string (nullable = true)
 |-- MentionDocLen: string (nullable = true)
 |-- MentionDocTone: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1type1Code: string (nullable = true)
 |-- Actor1type2Code: string (nullable = true)
 |-- Actor1type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCo

In [12]:
sample_joined.write.parquet('s3://bdcc-lab/joined_table.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_merged = sample_joined.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_merged.isnull().sum()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GlobalEventID               0
MentionTimeDate             0
MentionType                 0
SentenceID                  0
Actor1CharOffset            0
Actor2CharOffset            0
ActionCharOffset            0
MentionDocLen               0
MentionDocTone              0
Day                         0
Actor1Code               5244
Actor1CountryCode       27066
Actor1KnownGroupCode    43549
Actor1EthnicCode        43911
Actor1Religion1Code     42504
Actor1Religion2Code     43752
Actor1type1Code         20629
Actor1type2Code         42659
Actor1type3Code         44035
Actor2Code              16620
Actor2CountryCode       32768
Actor2KnownGroupCode    43524
Actor2EthnicCode        43955
Actor2Religion1Code     41575
Actor2Religion2Code     42561
Actor2type1Code         28437
Actor2type2Code         43012
Actor2type3Code         44058
IsRootEvent                 0
EventCode                   0
EventBaseCode               0
EventRootCode               0
QuadClass                   0
NumMention

In [26]:
df_merged.to_parquet('s3://bdcc-lab/df_merged.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_merged.to_pickle('df_merged.pkl')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  GlobalEventID MentionTimeDate  ... mention_source mention_article
0     813443217  20190101081500  ...           10.0             1.0
1     813443217  20190101080000  ...           10.0             1.0
2     813443217  20190101040000  ...           10.0             1.0
3     813449194  20190101053000  ...           10.0             1.0
4     813494031  20190101150000  ...           10.0             1.0

[5 rows x 39 columns]

## Reading merged parquet file

In [7]:
df_merged = spark.read.parquet('s3://bdcc-lab/df_merged.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_merged.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- GlobalEventID: string (nullable = true)
 |-- MentionTimeDate: string (nullable = true)
 |-- MentionType: string (nullable = true)
 |-- SentenceID: string (nullable = true)
 |-- Actor1CharOffset: string (nullable = true)
 |-- Actor2CharOffset: string (nullable = true)
 |-- ActionCharOffset: string (nullable = true)
 |-- MentionDocLen: string (nullable = true)
 |-- MentionDocTone: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- Actor1Religion1Code: string (nullable = true)
 |-- Actor1Religion2Code: string (nullable = true)
 |-- Actor1type1Code: string (nullable = true)
 |-- Actor1type2Code: string (nullable = true)
 |-- Actor1type3Code: string (nullable = true)
 |-- Actor2Code: string (nullable = true)
 |-- Actor2CountryCode: string (nullable = true)
 |-- Actor2KnownGroupCo

In [11]:
df_merged = df_merged.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_merged.to_csv('s3://bdcc-lab/bdcc_lab3_data.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…